In [1]:
pip install imgkit


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import imgkit
import os
import json

In [3]:
def highlight_max(s):
    is_max = s == s.max()
    is_second_best = s == s.nlargest(2).iloc[-1]  # Get the second largest value
    style = ['color: red' if v else '' for v in is_max]
    style = ['color: orange' if v else style[i] for i, v in enumerate(is_second_best)]
    return style

In [4]:

def create_mean_table(config_file, zoom_type):

    # Load the group_mask_class_1.csv file and the other table
    class_1_groups = pd.read_csv('groups_mask_class_1.csv')
    class_2_groups = pd.read_csv('groups_mask_class_2.csv')

    with open(config_file, 'r') as file:
        config = json.load(file)

    # Create a new dictionary to store the updated paths
    updated_paths = {}
    for key, path in config.items():
        # Concatenate the path with 'zoom_type_statistics.csv'
        updated_paths["Ground Truth"] = os.path.join(path, '1x_statistics.csv')
        updated_path = os.path.join(path, f'{zoom_type}_statistics.csv')
        updated_paths[key] = updated_path

    # Create a DataFrame for the means of class_1_data
    table_class_1 = pd.DataFrame({
        'Class': ['Maize'] * 5,
        'Group': ['False positives', "Small", "Medium", "Large", "Total"]
    })

    # Create a DataFrame for the means of class_2_data
    table_class_2 = pd.DataFrame({
        'Class': ['Weed'] * 5,
        'Group': ['False Positives',"Small", "Medium", "Large", "Total"]
    })

    for key, path in updated_paths.items():
        # Read the CSV file
        data = pd.read_csv(path)

        # Perform the merge based on the 'patch name' column
        class_1_data = pd.merge(class_1_groups, data, on='patch name', how='inner')
        class_2_data = pd.merge(class_2_groups, data, on='patch name', how='inner')

        # Calculate the mean of 'Dice_class1' for each group in class_1_data
        mean_dice_by_group_1 = class_1_data.groupby('Group')['Dice_class1'].mean().tolist()

        # Calculate the mean of 'Dice_class_2' for each group in class_2_data
        mean_dice_by_group_2 = class_2_data.groupby('Group')['Dice_class_2'].mean().tolist()

        # Calculate the total mean for class_1_data
        total_mean_class_1 = data['Dice_class1'].mean()

        # Calculate the total mean for class_2_data
        total_mean_class_2 = data['Dice_class_2'].mean()

        # Filter rows where Dice_class1 != 0 and Mask_class_1 == 0
        minor_flase_positives_1 = int(len(data[(data['Dice_class1'] > 0) & (data['Mask_class_1'] == 0)]))
        minor_flase_positives_2 = int(len(data[(data['Dice_class_2'] > 0) & (data['Mask_class_2'] == 0)]))


        column_name = f"DS_{key}"
        table_class_1[column_name] = [minor_flase_positives_1] + mean_dice_by_group_1 + [total_mean_class_1]
        table_class_2[column_name] = [minor_flase_positives_2] + mean_dice_by_group_2 + [total_mean_class_2]

    # Concatenate the two DataFrames
    result_table = pd.concat([table_class_1, table_class_2], ignore_index=True)

    # Apply conditional formatting to change text color for the highest and second-best mean values
    result_table = result_table.style.apply(highlight_max, subset=result_table.columns[3:], axis=1)

    # Show the table with the custom name
    print(f"Table: {zoom_type}")
    display(result_table)
    #html_table = result_table.render()

    # Save the HTML table as an image (you can customize the options)
    #imgkit.from_file('table.html', f'{key}.png')

    return result_table

def create_other_metrices(config_file, zoom_type):
    with open(config_file, 'r') as file:
        config = json.load(file)

    # Create a new dictionary to store the updated paths
    updated_paths = {}
    for key, path in config.items():
        # Concatenate the path with 'zoom_type_statistics.csv'
        updated_path = os.path.join(path, f'{zoom_type}_statistics.csv')
        updated_paths[key] = updated_path

    # Create a DataFrame for the means of class_1_data
    table_class_1 = pd.DataFrame({
        'Metrix': ['ssim', 'psnr']
    })

    for key, path in updated_paths.items():
        # Read the CSV file
        data = pd.read_csv(path)

        psnr = data['psnr'].mean().tolist()

        ssim = data['ssim'].mean().tolist()

        column_name = f"{key}"
        table_class_1[column_name] = [psnr, ssim]
    # Concatenate the two DataFrames
    result_table = table_class_1

    # Apply conditional formatting to change text color for the highest and second-best mean values
    result_table = result_table.style.apply(highlight_max, subset=result_table.columns[2:], axis=1)

    # Show the table with the custom name
    print(f"Table: {zoom_type}")
    display(result_table)

    return result_table

In [5]:
path_to_config = "/home/fatbardhf/SuperResolution_for_in-field_weed_images/configs/create_mean_tables.json"

print("For 2x:")
result_2x = create_mean_table(path_to_config, "2x")



For 2x:
Table: 2x


,Class,Group,DS_Ground Truth,DS_Bicubical,DS_SRCNN,DS_SRCNN_finetuned,DS_Real-ESRGAN,DS_Real-ESRGAN_finetuned,DS_HAT,DS_HAT_finetuned,DS_Filtering_Bilateral,DS_Filtering_Gaussian,DS_Filtering_Median,DS_Filtering_Non-local_means
0,Maize,False positives,52.000000,51.000000,46.000000,45.000000,70.000000,80.000000,45.000000,45.000000,110.000000,103.000000,76.000000,179.000000
1,Maize,Small,0.416215,0.408293,0.391492,0.401019,0.477921,0.387761,0.408938,0.415450,0.247556,0.311099,0.336271,0.120007
2,Maize,Medium,0.800823,0.807288,0.801028,0.805751,0.843279,0.787536,0.806069,0.808703,0.676829,0.724706,0.778906,0.331992
3,Maize,Large,0.636129,0.561788,0.583641,0.583099,0.838970,0.612202,0.596453,0.604210,0.413976,0.405484,0.490034,0.163138
4,Maize,Total,0.594497,0.593100,0.595975,0.602411,0.614822,0.537094,0.605190,0.608651,0.406467,0.448527,0.520253,0.168616
5,Weed,False Positives,66.000000,53.000000,60.000000,57.000000,56.000000,69.000000,57.000000,56.000000,84.000000,70.000000,64.000000,108.000000
6,Weed,Small,0.482526,0.452039,0.480246,0.479092,0.418031,0.368092,0.484415,0.484639,0.252706,0.296240,0.378554,0.064804
7,Weed,Medium,0.782463,0.749097,0.757875,0.760899,0.713500,0.730175,0.765497,0.763501,0.694866,0.700193,0.724807,0.476640
8,Weed,Large,0.893746,0.892558,0.892590,0.892631,0.868632,0.878734,0.891725,0.892704,0.866258,0.887647,0.890714,0.808934
9,Weed,Total,0.518449,0.505393,0.516509,0.519849,0.472134,0.437547,0.524101,0.524768,0.353481,0.389417,0.446708,0.184307


In [6]:
other_results_2x = create_other_metrices(path_to_config, "2x")

Table: 2x


,Metrix,Bicubical,SRCNN,SRCNN_finetuned,Real-ESRGAN,Real-ESRGAN_finetuned,HAT,HAT_finetuned,Filtering_Bilateral,Filtering_Gaussian,Filtering_Median,Filtering_Non-local_means
0,ssim,27.094131,28.159742,28.228216,23.664087,23.738163,29.221779,28.888797,24.097185,23.836280,24.412103,22.734333
1,psnr,0.803898,0.862269,0.863970,0.643697,0.621050,0.888487,0.880688,0.536796,0.547454,0.598574,0.415079


In [7]:
print("For 4x:")
result_4x = create_mean_table(path_to_config, "4x")
print(result_4x)

For 4x:
Table: 4x


,Class,Group,DS_Ground Truth,DS_Bicubical,DS_SRCNN,DS_SRCNN_finetuned,DS_Real-ESRGAN,DS_Real-ESRGAN_finetuned,DS_HAT,DS_HAT_finetuned,DS_Filtering_Bilateral,DS_Filtering_Gaussian,DS_Filtering_Median,DS_Filtering_Non-local_means
0,Maize,False positives,52.000000,94.000000,78.000000,70.000000,142.000000,209.000000,60.000000,55.000000,140.000000,122.000000,104.000000,186.000000
1,Maize,Small,0.416215,0.362340,0.318416,0.314371,0.336339,0.264176,0.405457,0.416421,0.297322,0.316752,0.346653,0.136325
2,Maize,Medium,0.800823,0.761079,0.730766,0.728092,0.767755,0.704770,0.794895,0.788280,0.651745,0.720557,0.761931,0.344962
3,Maize,Large,0.636129,0.447046,0.393859,0.390555,0.855249,0.563832,0.581017,0.568229,0.401318,0.416196,0.480270,0.177544
4,Maize,Total,0.594497,0.489467,0.481783,0.489609,0.458572,0.348936,0.572147,0.577610,0.374666,0.428090,0.477171,0.174819
5,Weed,False Positives,66.000000,60.000000,71.000000,70.000000,72.000000,85.000000,51.000000,55.000000,74.000000,67.000000,61.000000,108.000000
6,Weed,Small,0.482526,0.328142,0.369274,0.388646,0.179005,0.050879,0.421301,0.439476,0.173627,0.252328,0.291424,0.056404
7,Weed,Medium,0.782463,0.700027,0.727118,0.735462,0.609529,0.383193,0.744451,0.744008,0.628695,0.676231,0.691984,0.450925
8,Weed,Large,0.893746,0.889368,0.889776,0.888960,0.785039,0.778776,0.891124,0.890922,0.857043,0.882567,0.885623,0.772940
9,Weed,Total,0.518449,0.415120,0.436069,0.449807,0.295067,0.158973,0.488572,0.494736,0.299017,0.360704,0.391394,0.172143


In [8]:
other_results_4x = create_other_metrices(path_to_config, "4x")

Table: 4x


,Metrix,Bicubical,SRCNN,SRCNN_finetuned,Real-ESRGAN,Real-ESRGAN_finetuned,HAT,HAT_finetuned,Filtering_Bilateral,Filtering_Gaussian,Filtering_Median,Filtering_Non-local_means
0,ssim,23.512122,23.939778,23.880776,21.494494,21.187124,24.519390,24.425233,22.406932,22.900617,23.172161,21.931186
1,psnr,0.518239,0.582373,0.581433,0.419168,0.367217,0.629379,0.624136,0.368898,0.442277,0.472081,0.331929


In [9]:
print("For 8x:")
result_8x = create_mean_table(path_to_config, "8x")
print(result_8x)

For 8x:
Table: 8x


,Class,Group,DS_Ground Truth,DS_Bicubical,DS_SRCNN,DS_SRCNN_finetuned,DS_Real-ESRGAN,DS_Real-ESRGAN_finetuned,DS_HAT,DS_HAT_finetuned,DS_Filtering_Bilateral,DS_Filtering_Gaussian,DS_Filtering_Median,DS_Filtering_Non-local_means
0,Maize,False positives,52.000000,149.000000,140.000000,141.000000,233.000000,256.000000,121.000000,119.000000,181.000000,154.000000,154.000000,217.000000
1,Maize,Small,0.416215,0.275884,0.212246,0.237242,0.050304,0.000002,0.480100,0.479717,0.255859,0.271971,0.283252,0.087492
2,Maize,Medium,0.800823,0.637328,0.578235,0.585947,0.267976,0.100231,0.824091,0.812964,0.561951,0.620804,0.642787,0.338440
3,Maize,Large,0.636129,0.433607,0.360956,0.356987,0.163821,0.089789,0.662666,0.632491,0.374411,0.425285,0.449008,0.229493
4,Maize,Total,0.594497,0.359009,0.324326,0.331005,0.117670,0.041059,0.521696,0.516869,0.297126,0.346551,0.359555,0.157570
5,Weed,False Positives,66.000000,77.000000,92.000000,90.000000,97.000000,129.000000,37.000000,40.000000,87.000000,76.000000,75.000000,111.000000
6,Weed,Small,0.482526,0.172560,0.187205,0.183357,0.009067,0.004715,0.292615,0.284362,0.105378,0.147463,0.162399,0.031612
7,Weed,Medium,0.782463,0.594617,0.616524,0.611964,0.177143,0.084009,0.651520,0.649094,0.485501,0.566864,0.586127,0.353549
8,Weed,Large,0.893746,0.832792,0.850543,0.850084,0.583043,0.626521,0.871020,0.873178,0.751972,0.825051,0.829592,0.659851
9,Weed,Total,0.518449,0.286536,0.292535,0.290395,0.072668,0.045086,0.399428,0.391489,0.213396,0.265673,0.279708,0.129797


In [10]:
other_results_8x = create_other_metrices(path_to_config, "8x")

Table: 8x


,Metrix,Bicubical,SRCNN,SRCNN_finetuned,Real-ESRGAN,Real-ESRGAN_finetuned,HAT,HAT_finetuned,Filtering_Bilateral,Filtering_Gaussian,Filtering_Median,Filtering_Non-local_means
0,ssim,21.627831,21.568900,21.610864,19.920699,19.734064,22.250589,22.190131,21.348396,21.539207,21.591909,21.090554
1,psnr,0.306261,0.309609,0.308457,0.239101,0.231895,0.374178,0.367442,0.279053,0.297425,0.302204,0.260583
